# $\color{#19ABEA}{\text{Run gguf LLM models in Latest Version TextGen-webui :}}$

In [ ]:
#@title # $\color{#19ABEA}{\text{Download and install requirements}}$ {display-mode: "form"}
#@markdown $\color{#19ABEA}{\text{run this only once}}$

#@markdown `Note :no need to check these checkboxes unless u facing a problem`
#only if u want to remuve existing folder#!rm -rf /content/text-generation-webui
%cd /content
!git clone -b snapshot-2023-12-24 https://github.com/oobabooga/text-generation-webui
# !git clone -b V20231127 https://github.com/Troyanovsky/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install git+https://github.com/UWUplus/flask-cloudflared# the origin
!pip install -r extensions/openai/requirements.txt --upgrade
!apt-get -y install -qq aria2

reinstall_llama_cpp_python = False # @param {type:"boolean"}
if reinstall_llama_cpp_python :
  !pip uninstall -y llama-cpp-python
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir

#@markdown  `reinstalling flash attentionv2 may be needed  for some models like phi2..`
reinstall_flash_attention = False # @param {type:"boolean"}
if reinstall_flash_attention:
  !pip uninstall -y flash-attn
  !pip install --no-build-isolation flash-attn==2.3.0

from IPython.display import clear_output
clear_output()
print("Done")

In [ ]:
#@title # $\color{#19ABEA}{define funcs}$ {display-mode: "form"}
#@markdown  ________
#@markdown  ### run this only once
#@markdown  ________
#==================================================================
#==================================================================
#==================================================================
def downlink_to_dict(models_only=[]):
  if len(models_only) > 0 :
    dic ={}
    for model_link in models_only:
        splt =model_link.split(".")
        quantization_name = splt[len(splt)-2]
        dic[quantization_name] = model_link
    return dic
  return -1
def get_donwloadlink2(repolink_tree_main,quant=None):
    html = requests.get(repolink_tree_main)
    soup=bs(html.text, 'html.parser')
    all_downloads_link_ofcurrent_repo= soup.select("body > div.flex.min-h-screen.flex-col > main > div.container.relative.flex.flex-col.md\:grid.md\:space-y-0.w-full.md\:grid-cols-12.space-y-4.md\:gap-6.mb-16 > section > div:nth-child(4) > ul > li > a.group.col-span-4.flex.items-center.justify-self-end.truncate.text-right.font-mono.text-\[0\.8rem\].leading-6.text-gray-400.md\:col-span-2.xl\:pr-10")
    #filtering_doownload_link geeting only  models
    if quant!=None :
      models_only = [ i['href'] for i in all_downloads_link_ofcurrent_repo if '.gguf' in i['href'] and quant in i['href']]
      if len(models_only) >0 :
        return f"https://huggingface.co{models_only[0]}".replace("?download=true","")
      else :
        print("=========================================")
        print("The quantization choosed inot Available the only Quant Available is :")
        models_only = [ i['href'] for i in all_downloads_link_ofcurrent_repo if '.gguf' in i['href']]
        print("=========================================")
        print(models_only)
        return None
    else :
      models_only = [ i['href'] for i in all_downloads_link_ofcurrent_repo if '.gguf' in i['href']]
      return models_only

def get_filename(repo_name,dl_link):
  if dl_link is not None :
    return dl_link.replace(f"https://huggingface.co/{repo_name}/resolve/main/","").replace("?download=true","")
  else :
    return None
#===========================================================
#===========================================================
#===========================================================



In [ ]:
#@title #  $\color{#19ABEA}{Download Model}$ {display-mode: "form"}
from bs4 import BeautifulSoup as bs
import requests
#@markdown ## $\color{#19ABEA}{\text{Copy the RepoName here }}$

#@markdown  Example :`TheBloke/OpenHermes-2.5-neural-chat-7B-v3-1-7B-GGUF`
repo_name = "TheBloke/xDAN-L1-Chat-RL-v1-GGUF" # @param ["TheBloke/Mistral-7B-OpenOrca-GGUF", "TheBloke/zephyr-7B-alpha-GGUF", "TheBloke/zephyr-7B-beta-GGUF", "TheBloke/neural-chat-7B-v3-1-GGUF", "TheBloke/mistral-ft-optimized-1218-GGUF", "TheBloke/xDAN-L1-Chat-RL-v1-GGUF", "TheBloke/openchat-3.5-1210-starling-slerp-GGUF", "TheBloke/Loyal-Macaroni-Maid-7B-GGUF", "TheBloke/Valkyrie-V1-GGUF", "TheBloke/Sonya-7B-GGUF", "TheBloke/Starling-LM-7B-alpha-GGUF", "TheBloke/SOLAR-10.7B-v1.0-GGUF"] {allow-input: true}

#@markdown  in colab gpu (15Gvram) you can use :
#@markdown  - 13b model quantized upto Q5_K_M(context up to 4K)
#@markdown  - 7b model quantized upto Q8_0(context up to 16K)
flags="?not-for-all-audiences=true"
repo_link= f"https://huggingface.co/{repo_name}/tree/main{flags}"
# print(repo_link)
quant ="Q6_K" #@param ['Q2_K', 'Q3_K_L', 'Q3_K_M', 'Q3_K_S', 'Q4_0', 'Q4_1', 'Q4_K_M', 'Q4_K_S', 'Q5_0', 'Q5_1', 'Q5_K_M', 'Q5_K_S', 'Q6_K', 'Q8_0']
model_download_url =get_donwloadlink2(repo_link,quant=quant)
# print("dllink :" ,model_download_url)
model_file_name =get_filename(repo_name,model_download_url)
# print("filename : ",model_file_name)
if model_download_url is not None and model_file_name is not None :
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $model_download_url -d /content/text-generation-webui/models/ -o $model_file_name
else :
  print("======================")
  print("please check the repo name or availability or the quantization")

In [ ]:
#@title ##  $\color{#19ABEA}{RunOnly}$ {display-mode: "form"}
# %cd /content/text-generation-webui
#@markdown  $\color{#19ABEA}{\text{Enter ur Context size or let it Unckechet to use the default size}}$
params =" "
#@markdown _____
api =True # @param {type:"boolean"}
if api :params +=" --api --public-api --listen "
api_only_mode =False # @param {type:"boolean"}
if api_only_mode : params+=" --nowebui "
#@markdown _____
use_custom_context_len = True # @param {type:"boolean"}
context_len=4096*4 #@param
if use_custom_context_len : params += f" --n_ctx  {context_len} "
#@markdown _____
use_custom_params = True # @param {type:"boolean"}
custom_params = "" # @param {type:"string"}
if use_custom_params : params +=custom_params

#@markdown _____
# params
!python server.py   --share  --n-gpu-layers 100000  	--model $model_file_name   $params